In [114]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool

import pandas as pd
import random

In [115]:
import os

# Set the OpenAI API key globally
os.environ["OPENAI_API_KEY"] = "YOURKEY"
# Set Google API key and Search Engine ID directly in the notebook
os.environ["GOOGLE_API_KEY"] = "YOURKEYI"
os.environ["GOOGLE_SEARCH_ENGINE_ID"] = "YOURUD"

In [116]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:
    import os
    import time
    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")
    
    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": api_key, "cx": search_engine_id, "q": query, "num": num_results}

    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            return f"Error fetching {url}: {str(e)}"

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append({"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body})
        time.sleep(1)

    return enriched_results


In [117]:
def arxiv_search(query: str, max_results: int = 2) -> list:
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append({
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "published": paper.published.strftime("%Y-%m-%d"),
            "abstract": paper.summary,
            "pdf_url": paper.pdf_url,
        })

    return results


In [118]:
from autogen_core.tools import FunctionTool

def load_trump_tweets() -> str:
    """
    Reads the content from a pre-defined .txt file and returns it as a string.

    :return: Content of the file as a string
    """
    txt_file_path = r"C:\Users\gharb\trump_tweets.txt"  # Specify your file path here
    try:
        with open(txt_file_path, 'r', encoding='utf-8') as file:
            content = file.read()  # Read the entire content of the file
        return content
    except FileNotFoundError:
        return f"Error: File not found at {txt_file_path}"
    except Exception as e:
        return f"Error loading TXT file: {str(e)}"

# Wrap the function as a tool for AutoGen
load_trump_tweets_tool = FunctionTool(
    func=load_trump_tweets,
    description="Reads a predefined text file containing Trump tweets and returns its content as a string."
)


In [119]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.agents import UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


# Define the tools
google_search_tool = FunctionTool(
    google_search,
    description="Search Google for information, returns results with a snippet and body content"
)

arxiv_search_tool = FunctionTool(
    arxiv_search,
    description="Search Arxiv for papers related to a given topic, including abstracts"
)

# Define the tool
load_trump_tweets_tool = FunctionTool(
    load_trump_tweets,
    description="Load the content of a TXT file. Provide the full file path."
   
)

# Define agents

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    system_message="""
    You are a planning agent.
    You are the central planning agent responsible for orchestrating a multi-agent marketing workflow. Your task is to logically distribute tasks among agents based on the required output and the nature of the task. Assign agents dynamically to ensure efficiency and relevance. Your objective is to optimize task allocation, minimize redundancy, and ensure all agents collaborate effectively toward the final goal. all agents should participate in the tasks.
    Your team members are:
    Google Search Agent: Perform Research in Google to Gather Up-to-date infromations, insights, statstics, keywords, ideas, trends. you will ask for outup it depend to the task.
    Arvix Search Agent: This agents searches for informations, articles, litteratures to support the content production.  you will ask for outup it depend to the task. also you can can override/bypass him if you find his particpation not necessary.
    Trump Tweets: Important This Agent Will Process Trump Tweets and give Insights to Trump Agent to mimic the language of donald trump.
    Trump Agent:  This Agent Should Write Content Like Donald Trump, Copy His Personality and Style.
    Social Media Agent: This agent promotes content created by the Trump Agent by crafting engaging hooks, creating suspense, and suggesting design ideas to drive users to external platforms to read the full content.

    When assigning tasks, use this format:
    1. <agent> : <task>
    2. <agent> : <task>
    3. <agent> : <task>
    4. <agent> : <task>
    5. <agent> : <task>

    Do not use the termination keyword ('TERMINATE') in your intermediate instructions. Include it only after all subtasks are completed successfully. all agents should participate. After ALL Task Completed Summarize The Finding in Structured Output
      """,
)


google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Google for information and provide enriched content",
    system_message="You are a helpful assistant skilled in performing web searches, Findind Informations, Statstiques, Interviews. Your Output Will Help the Content Creator Agentn to produce content."
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Arxiv for academic papers",
    system_message="You are a helpful assistant skilled in finding relevant academic papers."
)


trump_tweets = AssistantAgent(
    name="trump_tweets_agent",
    tools=[load_trump_tweets_tool], 
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that analyzes text content from .txt files and provides insights.",
    system_message=("Your task is to analyze a dataset of Donald Trump’s tweets to fully understand his unique style of communication. Identify his common phrases, tone, and rhetorical strategies. Pay attention to his frequent use of simple, impactful language, repetitive emphasis on key points, and superlatives like 'great,' 'tremendous,' and 'fantastic.' Note how he often uses bold, declarative statements, rhetorical questions, and a conversational tone. Focus on how he combines confidence with controversy, addressing topics with a mix of assertiveness and exaggeration. Use this analysis to generate responses or content that mimic his characteristic voice and style accurately."
)
)
trump_agent = AssistantAgent(
    name="trump_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Generate a structured content based on the data collected, Or any other type of Content",
 system_message=(
        "Wait for the trump tweets agent send you the content. You are Donald Trump, the 45th President of the United States. Your task is to use the infromation that you receive from arvix search agent, google search agent to create the required type of content and to use the tweets you will receive from trump tweets agent to use as a second ressource to help you mimic donald trump writting style. convert content into your distinct style—confident, bold, and persuasive, Speak with authority, using simple phrases and repetition to emphasize key points. Include terms like 'believe me,' 'great,' 'fantastic,' 'tremendous,' and 'the best, Your tone should be unapologetic, assertive, and captivating, with sweeping statements that grab attention, Draw inspiration from your well-known speaking patterns and tweets to craft responses that are unmistakably Trump-like, After transforming the content, ensure it is structured, impactful, and ready for use."
    )
)

social_media_agent = AssistantAgent(
    name="Social_Media_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description=(
        "Promote content generated by other agents, such as the Trump Agent, "
        "by crafting engaging hooks, creating suspense, and suggesting creative ideas for designs. "
        "This agent's primary goal is to drive traffic to the full content without directly sharing it."
    ),
    system_message=(
        "You are a Social Media Expert. Your role is to promote content indirectly by crafting compelling hooks, "
        "suggesting intriguing design ideas, and finding ways to create suspense that will push people to visit the source. "
        "Your outputs should include suggestions for captions, visuals, and strategies to generate curiosity. "
        "You can analyze the tone of the Trump Agent's content and align your strategy accordingly. "
        "Remember: You are not sharing the content but driving people to engage with it outside of social platforms. End your response with 'TERMINATE'."
    )
)


In [120]:
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination, MaxMessageTermination
from autogen_core import CancellationToken
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=5000)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, google_search_agent, arxiv_search_agent, trump_tweets, trump_agent, social_media_agent],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    termination_condition=termination,
)


In [121]:
task = "a speech to the world after potates price reached 500$ for 1kg"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))


---------- user ----------
a speech to the world after potates price reached 500$ for 1kg
---------- PlanningAgent ----------
To create a compelling speech on the situation where potato prices have skyrocketed to $500 for 1kg, I will first gather insights and context on the topic. I will assign the tasks to the agents accordingly.

1. Google Search Agent: Perform research to gather up-to-date information about potato pricing trends, causes behind the sudden price spike, and potential economic impacts.
2. Arvix Search Agent: Search for articles, studies, or literature discussing the agricultural economics related to potato prices and supply chain disruptions.
3. Trump Tweets: Analyze Trump’s tweets related to food prices and agriculture for language style insights.
4. Trump Agent: Write a speech using Donald Trump's style to address the global audience concerning the potato price crisis.
5. Social Media Agent: Create promotional content and hooks to share the speech on social media plat

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='a speech to the world after potates price reached 500$ for 1kg', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=411, completion_tokens=178), content="To create a compelling speech on the situation where potato prices have skyrocketed to $500 for 1kg, I will first gather insights and context on the topic. I will assign the tasks to the agents accordingly.\n\n1. Google Search Agent: Perform research to gather up-to-date information about potato pricing trends, causes behind the sudden price spike, and potential economic impacts.\n2. Arvix Search Agent: Search for articles, studies, or literature discussing the agricultural economics related to potato prices and supply chain disruptions.\n3. Trump Tweets: Analyze Trump’s tweets related to food prices and agriculture for language style insights.\n4. Trump Agent: Write a speech using Donald Trump's style to address t